In [1]:
# mlp for regression
from numpy import sqrt
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# load the dataset
df = read_csv("Memory_SolarTrainingData.csv", index_col=0)

In [3]:
df.tail()

,onefourtyfive_ago,onethirty_ago,onefifteen_ago,one_ago,fourtyfive_ago,thirty_ago,fifteen_ago,temp,feelslike,dew,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,year,month,day,time,Percent Output
2824,0.44480,0.36112,0.29264,0.33872,0.36496,0.18032,0.15104,90.3,96.7,71.1,...,20.1,9.9,141.0,0.5,1.0,2018,8,29,19.0,0.07072
2825,0.36496,0.18032,0.15104,0.07072,0.08704,0.06112,0.00672,84.6,90.7,72.4,...,20.1,9.9,20.0,0.1,0.0,2018,8,29,20.0,-0.00416
2844,0.58128,0.49440,0.47568,0.63408,0.61072,0.56400,0.60672,96.2,99.3,65.4,...,48.7,9.9,817.0,2.9,8.0,2018,8,30,15.0,0.61072
2845,0.61072,0.56400,0.60672,0.61072,0.60624,0.60432,0.60320,96.4,98.5,63.7,...,14.2,9.9,707.0,2.5,7.0,2018,8,30,16.0,0.62064
2871,0.58672,0.57408,0.56256,0.53728,0.46080,0.48064,0.45312,97.4,99.1,62.8,...,11.3,9.9,405.0,1.5,4.0,2018,8,31,18.0,0.41600


In [4]:
# split into past, current, and future data
pastWeatherAndSupply = df.iloc[:-2]

currentWeather = df.iloc[-2]
currentWeather = currentWeather.drop(['Percent Output']) #dropping supply column for testing
currentWeather = currentWeather.values[:].reshape((1,27))

nextHourWeather = df.iloc[-1]
nextHourWeather = nextHourWeather.drop(['Percent Output']) #dropping supply column for testing
nextHourWeather = nextHourWeather.values[:].reshape((1,27))

In [6]:
print(currentWeather)

[[6.1072e-01 5.6400e-01 6.0672e-01 6.1072e-01 6.0624e-01 6.0432e-01
  6.0320e-01 9.6400e+01 9.8500e+01 6.3700e+01 3.4260e+01 0.0000e+00
  0.0000e+00 0.0000e+00 1.6100e+01 4.6000e+00 9.2000e+01 1.0132e+03
  1.4200e+01 9.9000e+00 7.0700e+02 2.5000e+00 7.0000e+00 2.0180e+03
  8.0000e+00 3.0000e+01 1.6000e+01]]


In [7]:
print(nextHourWeather)

[[5.8672e-01 5.7408e-01 5.6256e-01 5.3728e-01 4.6080e-01 4.8064e-01
  4.5312e-01 9.7400e+01 9.9100e+01 6.2800e+01 3.2190e+01 0.0000e+00
  0.0000e+00 0.0000e+00 1.6100e+01 1.0600e+01 1.3800e+02 1.0096e+03
  1.1300e+01 9.9000e+00 4.0500e+02 1.5000e+00 4.0000e+00 2.0180e+03
  8.0000e+00 3.1000e+01 1.8000e+01]]


In [102]:
# split into input and output columns
X, y = pastWeatherAndSupply.values[:, :-1], pastWeatherAndSupply.values[:, -1]

In [103]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(401, 27) (101, 27) (401,) (101,)


In [104]:
# determine the number of input features
n_features = X_train.shape[1]
print(n_features)

27


In [105]:
# define model
model = Sequential()
model.add(Dense(10, activation='sigmoid', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))

In [106]:
# compile the model
model.compile(optimizer='adam', loss='mse')

In [107]:
# fit the model
model.fit(X_train, y_train, epochs=300, batch_size=10, verbose=0)

In [108]:
# evaluate the model
error = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, sqrt(error)))

MSE: 0.044, RMSE: 0.209


In [109]:
# Making a prediction for integration with control code
# IMPROVEMENT AREA: Sometimes the model will predict a number greater than 1. This should not happen.
current_prediction = model.predict(currentWeather)
if (current_prediction < 0):
    current_prediction = 0
print('Predicted: %.3f' % current_prediction)

future_prediction = model.predict(nextHourWeather)
if (current_prediction < 0):
    current_prediction = 0
print('Predicted: %.3f' % future_prediction)

1/1 [==============================] - 0s 70ms/step
Predicted: 0.351
1/1 [==============================] - 0s 22ms/step
Predicted: 0.351


In [110]:
# Return predictions in a list
#return([current_prediction, future_prediction])